# Importando bibliotecas e arquivos

In [ ]:
import random
import csv
import os
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
from plotly import graph_objects as go
from google.colab import files
from sklearn.covariance import EmpiricalCovariance, MinCovDet

import warnings

# Ignorando warnings de runtime da cov robusta

warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

smpsinal =np.load('/content/drive/MyDrive/Filtro_casado_estocastico/Dados/SmpSinal.npy')
smpruido =np.load('/content/drive/MyDrive/Filtro_casado_estocastico/Dados/SmpRuido.npy')
EneStile=np.load('/content/drive/MyDrive/Filtro_casado_estocastico/Dados/TileModSinal.npy')
EneRtile=np.load('/content/drive/MyDrive/Filtro_casado_estocastico/Dados/TileModRuido.npy')
EneStmdb_ch=np.load('/content/drive/MyDrive/Filtro_casado_estocastico/Dados/TMDBSinalCh.npy')
EneRtmdb_ch=np.load('/content/drive/MyDrive/Filtro_casado_estocastico/Dados/TMDBRuidoCh.npy')

Mounted at /content/drive


# Definindo as funções

### Função para branqueamento

In [ ]:
def bran(c):
  """
  Branqueamento
  """

  D, V = np.linalg.eig(c)

  # reordenando autovalores e autovetores pra ficar igual ao matlab
  indices = np.argsort(D)
  D = D[indices]
  D = D.real
  V = V[:, indices]

  D = D**(-0.5) # esta é a primeira parte do calculo do W
  D = D*np.identity(7) # agrupei o array D em uma matriz diagonal

  W = np.matmul(D,V.T) # produto matricial

  return W

### Função que calcula o pulso médio

In [ ]:
def mp(nf):
  """
  Returns the medium pulse
  """
  a0 = 0.0
  a1 = 0.0
  a2 = 0.0
  a3 = 0.0
  a4 = 0.0
  a5 = 0.0
  a6 = 0.0

  for i in range(len(nf)):
    a0 = a0 + nf[i][0]
    a1 = a1 + nf[i][1]
    a2 = a2 + nf[i][2]
    a3 = a3 + nf[i][3]
    a4 = a4 + nf[i][4]
    a5 = a5 + nf[i][5]
    a6 = a6 + nf[i][6]

  a0 = a0/len(nf)
  a1 = a1/len(nf)
  a2 = a2/len(nf)
  a3 = a3/len(nf)
  a4 = a4/len(nf)
  a5 = a5/len(nf)
  a6 = a6/len(nf)

  sm = []

  sm.append(a0)
  sm.append(a1)
  sm.append(a2)
  sm.append(a3)
  sm.append(a4)
  sm.append(a5)
  sm.append(a6)

  div = max(sm)
  for i in range(7):
    sm[i] = sm[i]/div
  
  return sm

### Função para cov robusta

In [ ]:
def robust_cov(A):
  """
  retorna a covariância robusta
  """

  robust_cov = MinCovDet().fit(A.T).covariance_

  return robust_cov


### Função para PCA

In [ ]:
def pca(A):
 """ performs principal components analysis 
     (PCA) on the n-by-p data matrix A
     Rows of A correspond to observations, columns to variables. 

 Returns :  
  coeff :
    is a p-by-p matrix, each column containing coefficients 
    for one principal component.
  score : 
    the principal component scores; that is, the representation 
    of A in the principal component space. Rows of SCORE 
    correspond to observations, columns to components.
  latent : 
    a vector containing the eigenvalues 
    of the covariance matrix of A.
 """
 # computing eigenvalues and eigenvectors of covariance matrix
 M = (A-np.mean(A.T,axis=1)).T # subtract the mean (along columns)
 
 [latent,coeff] = np.linalg.eig(robust_cov(M)) # attention:not always sorted

 # reordenando autovalores e autovetores
 indices = np.argsort(latent)[::-1]
 latent = latent[indices]
 coeff = coeff[:, indices]


 score = np.dot(coeff.T,M) # projection of the data in the new space
 return coeff,score,latent

### Função para aplicação do filtro

In [ ]:
def StochFilter(sinalTes, ruidoTes, ruidoDes, W, coeff, latent, mEstimacao):
  """
  Filtering
  """

  # Calculando os parametros

  rRuido = np.matmul(ruidoTes, W.T)
  rRuido = np.matmul(rRuido, coeff)

  rSinal = np.matmul(sinalTes, W.T)
  rSinal = np.matmul(rSinal, coeff)

  variancia = np.var(ruidoDes)

  No = variancia*2

  N = 7

  lbd = latent

  h1 = np.zeros((7,7), dtype=float) 
  h2 = np.zeros((7,7), dtype=float) 

  aux1 = []
  aux2 = []

  for i in range(N):# de 1 ate o numero de pca
      h1 = h1 + (lbd[i]/(lbd[i] + variancia)) * np.matmul(np.transpose(np.matrix(coeff[:][i])), np.matrix(coeff[:][i]))
      h2 = h2 + (1/(lbd[i] + variancia)) * np.matmul(np.transpose(np.matrix(coeff[:][i])), np.matrix(coeff[:][i]))

  # Componente Estocastica

  IrRuido = np.zeros((len(ruidoTes),1), dtype=float) 
  IrSinal = np.zeros((len(ruidoTes),1), dtype=float) 

  for ev in range(len(ruidoTes)):
    aux = np.transpose(np.matmul(rRuido[ev][:] , np.matrix(coeff[:][0:N])))
    IrRuido[ev] = (1/No)*(np.matmul(np.matmul(rRuido[ev][:] , np.matrix(coeff[:][0:N])) , np.matmul(h1 , aux)))

  for ev in range(len(sinalTes)):
    aux = np.transpose(np.matmul(rSinal[ev][:] , np.matrix(coeff[:][0:N])))
    IrSinal[ev] = (1/No)*(np.matmul(np.matmul(rSinal[ev][:] , np.matrix(coeff[:][0:N])) , np.matmul(h1 , aux)))

  # Componente Deterministica

  IdRuido = np.zeros((len(ruidoTes),1), dtype=float) 
  IdSinal = np.zeros((len(ruidoTes),1), dtype=float)

  for ev in range(len(ruidoTes)):
    aux = np.transpose(np.matmul(rRuido[ev][:] , np.matrix(coeff[:][0:N])))
    IdRuido[ev] = np.matmul(np.matmul(mEstimacao , np.matrix(coeff[:][0:N])) , np.matmul(h2 , aux))

  for ev in range(len(sinalTes)):
    aux = np.transpose(np.matmul(rSinal[ev][:] , np.matrix(coeff[:][0:N])))
    IdSinal[ev] = np.matmul(np.matmul(mEstimacao , np.matrix(coeff[:][0:N])) , np.matmul(h2 , aux))

  FCestRuido = IdRuido + IrRuido
  FCestSinal = IdSinal + IrSinal 

  # Etapa de estimaçaão de amplitude

  b1 = np.matmul(np.matrix(coeff[:][0:N]) , np.matrix(np.transpose(coeff[:][0:N])))
  b2 = (1/No) * np.matmul(np.matrix(coeff[:][0:N]) , np.matmul(np.matrix(h1) , np.transpose(np.matrix(coeff[:][0:N]))))
  b3 = np.matmul(np.matmul(mEstimacao , np.transpose(np.matrix(coeff[:][0:N]))) ,  np.matmul(np.matrix(h2) , np.transpose(np.matrix(coeff[:][0:N]))))

  ampRuido = np.zeros((len(ruidoTes),1), dtype=float) 
  ampSinal = np.zeros((len(ruidoTes),1), dtype=float)

  a = (1/No) * np.matmul(np.matmul(mEstimacao , np.transpose(np.matrix(coeff[:][0:N]))),  np.matmul(np.matrix(h1) , np.transpose(np.matmul(mEstimacao , np.transpose(np.matrix(coeff[:][0:N]))))))
  b = np.matmul(np.matmul(mEstimacao , np.transpose(np.matrix(coeff[:][0:N]))),  np.matmul(np.matrix(h2) , np.transpose(np.matmul(mEstimacao , np.transpose(np.matrix(coeff[:][0:N]))))))

  cs=0;
  cr=0;

  for i in range(len(sinalTes)):
    # ra = np.matmul(b , b) + 4 * np.matmul(a , FCestSinal[i][:]) 
    ra = b*b+4*a*FCestSinal[i]
    if ra < 0:
      ra = 0
      cs = cs + 1
    ampSinal[i] = (- b + np.sqrt(ra))/(2*a)
  
  for i in range(len(ruidoTes)):
    # ra = np.matmul(b , b) + 4 * np.matmul(a , FCestRuido[i][:])
    ra = b*b+4*a*FCestRuido[i]
    if ra < 0:
      ra = 0
      cr = cr + 1
    ampRuido[i] = (- b + np.sqrt(ra))/(2*a)


  return FCestSinal, FCestRuido, ampSinal, ampRuido,

### Função pra curva ROC

In [ ]:
def roc(pontos, sinal, ruido):
  """ calcula os parametros da curva ROC
  (Receiver Operating Characteristic Curve)

 Retorna :  
  FA :
    falso alarme
  PD :
    probabilidade de detecção
  
  """
  pmin = min(ruido)
  pmax = max(ruido)

  psoma = (pmax+abs(pmin))/pontos
  patamar = pmin
  PD = []
  FA = []
  pd = 0
  fa = 0

  for i in range(pontos):
    for j in range(len(ruido)):
      if sinal[j] > patamar:
        pd = pd + 1
      if ruido[j] > patamar:
        fa = fa + 1
    
    pd = pd*100/len(sinal)
    fa = fa*100/len(ruido)
    PD.append(pd)
    FA.append(fa)
    pd = 0
    fa = 0
    patamar = patamar + psoma

  FA = np.array(FA)
  PD = np.array(PD)

  return FA, PD

### Função para ponto de PD = 98%

In [ ]:
def p98(FA, PD):
  """
  retorna os valores do ponto (x,y) da curva mais proximo de 98%
  """

  x = []
  y = []

  for i in range(len(PD)):
    if PD[i]>=0.98:
      y.append(PD[i])
      x.append(FA[i])
  x = min(x)
  y = min(y)
  return x, y


# Testes

In [ ]:
# Selecionando o lado e módulo
# side = 1
# mod = 57
EneSTMDB=EneStmdb_ch.sum(axis=2)
EneRTMDB=EneRtmdb_ch.sum(axis=2)
base_fpr = np.linspace(0, 1, 1000)

size_kfold=10  #escolhendo o numero de kfold - normalmente de faz com 10 
kf=KFold(n_splits=size_kfold,shuffle=False)

data = [[],[],[],[],[],[],[],[],[],[]]

# pontos = open("pontos.txt",  'w+')

for side in range(0,1):
  for mod in range(0,1):
    # Calculando o pedestal para cada canal

    ped = np.array([0.0, 0.0, 0.0, 0.0]).astype(np.float)

    for i in range(np.size(ped)):
      for j in range(np.size(smpruido, 4)):
        ped[i] = ped[i] + smpruido[side][mod][i][4][j]

    for i in range(4):
      ped[i] = ped[i]/np.size(smpruido, 4)

    # Separando os dados por canal

    ruido0 = []
    ruido1 = []
    ruido2 = []
    ruido3 = []
    for j in range(np.size(smpruido, 4)):
      aux0 = []
      aux1 = []
      aux2 = []
      aux3 = []
      for i in range(np.size(smpruido, 3)):
        aux0.append(smpruido[side][mod][0][i][j])
        aux1.append(smpruido[side][mod][1][i][j])
        aux2.append(smpruido[side][mod][2][i][j])
        aux3.append(smpruido[side][mod][3][i][j])
      ruido0.append(aux0)
      ruido1.append(aux1)
      ruido2.append(aux2)
      ruido3.append(aux3)


    sinal0 = []
    sinal1 = []
    sinal2 = []
    sinal3 = []
    for j in range(np.size(smpsinal, 4)):
      aux0 = []
      aux1 = []
      aux2 = []
      aux3 = []
      for i in range(np.size(smpsinal, 3)):
        aux0.append(smpsinal[side][mod][0][i][j])
        aux1.append(smpsinal[side][mod][1][i][j])
        aux2.append(smpsinal[side][mod][2][i][j])
        aux3.append(smpsinal[side][mod][3][i][j])
      sinal0.append(aux0)
      sinal1.append(aux1)
      sinal2.append(aux2)
      sinal3.append(aux3)

    # Separando o conjunto de dados para treino (80%) e para teste (20%)

    ruido0 = np.array(ruido0)
    ruido1 = np.array(ruido1)
    ruido2 = np.array(ruido2)
    ruido3 = np.array(ruido3)

    sinal0 = np.array(sinal0)
    sinal1 = np.array(sinal1)
    sinal2 = np.array(sinal2)
    sinal3 = np.array(sinal3)

    ruido0 = ruido0 - ped[0]
    ruido1 = ruido1 - ped[1]
    ruido2 = ruido2 - ped[2]
    ruido3 = ruido3 - ped[3]

    sinal0 = sinal0 - ped[0]
    sinal1 = sinal1 - ped[1]
    sinal2 = sinal2 - ped[2]
    sinal3 = sinal3 - ped[3]

    ruidob=ruido1.T
    ruido0pd=pd.DataFrame(ruido0.T)
    sinal0pd=pd.DataFrame(sinal0.T) 

    ruido1pd=pd.DataFrame(ruido1.T)
    sinal1pd=pd.DataFrame(sinal1.T) 

    ruido2pd=pd.DataFrame(ruido2.T)
    sinal2pd=pd.DataFrame(sinal2.T) 

    ruido3pd=pd.DataFrame(ruido3.T)
    sinal3pd=pd.DataFrame(sinal3.T) 

    flag=0
    tprs = []
    tprs_SS = []
    for train_index, test_index in kf.split(ruidob[0]):
      
      print('Lado:',side, 'modulo:',mod, 'Numero kfold:', flag)
      
      ruidoDes0 = ruido0pd.iloc[:,train_index]
      ruidoTes0 = ruido0pd.iloc[:,test_index]
      sinalDes0 = sinal0pd.iloc[:,train_index]
      sinalTes0 = sinal0pd.iloc[:,test_index]

      ruidoDes1 = ruido1pd.iloc[:,train_index]
      ruidoTes1 = ruido1pd.iloc[:,test_index]
      sinalDes1 = sinal1pd.iloc[:,train_index]
      sinalTes1 = sinal1pd.iloc[:,test_index]

      ruidoDes2 = ruido2pd.iloc[:,train_index]
      ruidoTes2 = ruido2pd.iloc[:,test_index]
      sinalDes2 = sinal2pd.iloc[:,train_index]
      sinalTes2 = sinal2pd.iloc[:,test_index]

      ruidoDes3 = ruido3pd.iloc[:,train_index]
      ruidoTes3 = ruido3pd.iloc[:,test_index]
      sinalDes3 = sinal3pd.iloc[:,train_index]
      sinalTes3 = sinal3pd.iloc[:,test_index]

      zero1=np.zeros(len(test_index))
      um1=np.ones(len(test_index))

      true_value=np.concatenate((um1,zero1),axis=0)
      

      EneTMDB_S_test=pd.DataFrame(EneSTMDB[side][mod]).iloc[test_index]
      EneTMDB_R_test=pd.DataFrame(EneRTMDB[side][mod]).iloc[test_index]

      EneStile_test=pd.DataFrame(EneStile[side][mod]).iloc[test_index]
      EneRtile_test=pd.DataFrame(EneRtile[side][mod]).iloc[test_index]

      # Matriz de covariância do ruído antes do branqueamento

      c0 = np.cov(ruidoDes0)
      c1 = np.cov(ruidoDes1)
      c2 = np.cov(ruidoDes2)
      c3 = np.cov(ruidoDes3)

      # Matriz de branqueamento

      W0 = bran(c0)
      W1 = bran(c1)
      W2 = bran(c2)
      W3 = bran(c3)

      # Filtrando os dados para um corte de energia

      F0 = []
      F1 = []
      F2 = []
      F3 = []

      # for i in range(len(sinalDes0.T)):

        # if (EneStmdb_ch[side][mod][0][i] + EneStmdb_ch[side][mod][1][i] + EneStmdb_ch[side][mod][2][i] + EneStmdb_ch[side][mod][3][i])>3000:
          
        #   F.append(sinalDes.iloc[:,i])
      A = 150#np.mean(EneStmdb_ch[side][mod][0][:])*4
      B = 150#np.mean(EneStmdb_ch[side][mod][1][:])*4
      C = 150#np.mean(EneStmdb_ch[side][mod][2][:])*2
      D = 150#np.mean(EneStmdb_ch[side][mod][3][:])*2

      sinalDes0 = np.array(sinalDes0.T)
      sinalDes1 = np.array(sinalDes1.T)
      sinalDes2 = np.array(sinalDes2.T)
      sinalDes3 = np.array(sinalDes3.T)
      
      for i in range(len(sinalDes0)):
        if (EneStmdb_ch[side][mod][0][i] > A):
          if (np.max(sinalDes0[i][:]) < 200):
            F0.append(sinalDes0[i][:])

      for i in range(len(sinalDes1)):
        if (EneStmdb_ch[side][mod][1][i] > B):
          if (np.max(sinalDes1[i][:]) < 200):
            F1.append(sinalDes1[i][:])

      for i in range(len(sinalDes2)):
        if (EneStmdb_ch[side][mod][2][i] > C):
          if (np.max(sinalDes2[i][:]) < 200):
            F2.append(sinalDes2[i][:])

      for i in range(len(sinalDes3)):
        if (EneStmdb_ch[side][mod][3][i] > D):
          if (np.max(sinalDes3[i][:]) < 200):
            F3.append(sinalDes3[i][:])
            
      # if len(F0) == 0:
      #   F0 = np.zeros((len(sinalDes0.T),7))
      
      # if len(F1) == 0:
      #   F1 = np.zeros((len(sinalDes0.T),7))
      
      # if len(F2) == 0:
      #   F2 = np.zeros((len(sinalDes0.T),7))
      
      # L3 = len(F3)
      # if len(F3) == 0:
      #   F3 = np.zeros((len(sinalDes0.T),7))

      NF0 = F0
      NF1 = F1
      NF2 = F2
      NF3 = F3

      for i in range(len(F0)):
        div = max(F0[i])
        for j in range(7):
          NF0[i][j] = F0[i][j]/div

      for i in range(len(F1)):
        div = max(F1[i])
        for j in range(7):
          NF1[i][j] = F1[i][j]/div

      for i in range(len(F2)):
        div = max(F2[i])
        for j in range(7):
          NF2[i][j] = F2[i][j]/div

      for i in range(len(F3)):
        div = max(F3[i])
        for j in range(7):
          NF3[i][j] = F3[i][j]/div

      # Obtendo o pulso médio dos dados normalizados e filtrados

      sm0 = mp(NF0)
      sm1 = mp(NF1)
      sm2 = mp(NF2)
      sm3 = mp(NF3)

      NF0 = np.array(NF0)
      NF1 = np.array(NF1)
      NF2 = np.array(NF2)
      NF3 = np.array(NF3)

      # PCA e obtenção dos sinais médios

      pca_data0 = np.matmul(NF0, W0.T)
      coeff0, score0, latent0 = pca(pca_data0)

      pca_data1 = np.matmul(NF1, W1.T)
      coeff1, score1, latent1 = pca(pca_data1)

      pca_data2 = np.matmul(NF2, W2.T)
      coeff2, score2, latent2 = pca(pca_data2)

      pca_data3 = np.matmul(NF3, W3.T)
      coeff3, score3, latent3 = pca(pca_data3)

      N = 7

      mFC0 = np.matmul(sm0, W0.T)
      mEstimacao0 = np.matmul(mFC0, coeff0)

      mFC1 = np.matmul(sm1, W1.T)
      mEstimacao1 = np.matmul(mFC1, coeff1)

      mFC2 = np.matmul(sm2, W2.T)
      mEstimacao2 = np.matmul(mFC2, coeff2)

      mFC3 = np.matmul(sm3, W3.T)
      mEstimacao3 = np.matmul(mFC3, coeff3)

      # Cálculo das componentes do filtro
      
      sinalTes0 = np.array(sinalTes0.T)
      ruidoTes0 = np.array(ruidoTes0.T)
      ruidoDes0 = np.array(ruidoDes0.T)

      sinalTes1 = np.array(sinalTes1.T)
      ruidoTes1 = np.array(ruidoTes1.T)
      ruidoDes1 = np.array(ruidoDes1.T)

      sinalTes2 = np.array(sinalTes2.T)
      ruidoTes2 = np.array(ruidoTes2.T)
      ruidoDes2 = np.array(ruidoDes2.T)

      sinalTes3 = np.array(sinalTes3.T)
      ruidoTes3 = np.array(ruidoTes3.T)
      ruidoDes3 = np.array(ruidoDes3.T)

      FCestSinal0, FCestRuido0, ampSinal0, ampRuido0 = StochFilter(sinalTes0, ruidoTes0, ruidoDes0, W0, coeff0, latent0, mEstimacao0)
      FCestSinal1, FCestRuido1, ampSinal1, ampRuido1 = StochFilter(sinalTes1, ruidoTes1, ruidoDes1, W1, coeff1, latent1, mEstimacao1)
      FCestSinal2, FCestRuido2, ampSinal2, ampRuido2 = StochFilter(sinalTes2, ruidoTes2, ruidoDes2, W2, coeff2, latent2, mEstimacao2)
      FCestSinal3, FCestRuido3, ampSinal3, ampRuido3 = StochFilter(sinalTes3, ruidoTes3, ruidoDes3, W3, coeff3, latent3, mEstimacao3)

      FCestSinal = np.array(ampSinal0) + np.array(ampSinal1) + np.array(ampSinal2) + np.array(ampSinal3)
      FCestRuido = np.array(ampRuido0) + np.array(ampRuido1) + np.array(ampRuido2) + np.array(ampRuido3)
      
  

      saida_filtro=np.concatenate((FCestSinal,FCestRuido),axis=0)
      
      saida_TMDB=np.concatenate((EneTMDB_S_test,EneTMDB_R_test),axis=0)

      saida_Tile=np.concatenate((EneRtile_test,EneStile_test),axis=0)

      data[flag] = [[saida_filtro], [saida_Tile]] 
      
      fpr, tpr, _ =metrics.roc_curve(true_value,saida_filtro)
      tpr = np.interp(base_fpr, fpr, tpr)
      tpr[0] = 0.0
      tprs.append(tpr)

      fpr_SS, tpr_SS, _ =metrics.roc_curve(true_value,saida_TMDB)
      tpr_SS = np.interp(base_fpr, fpr_SS, tpr_SS)
      tpr_SS[0] = 0.0
      tprs_SS.append(tpr_SS)

      flag += 1


    tprs2 = np.array(tprs)
    mean_tprs2 = tprs2.mean(axis=0)
    std_tprs2 = tprs2.std(axis=0)
    tprs_upper = np.minimum(mean_tprs2 + std_tprs2, 1)
    tprs_lower = mean_tprs2 - std_tprs2
    mean_auc = metrics.auc(base_fpr, mean_tprs2)

    xFCE, yFCE = p98(base_fpr, mean_tprs2)


    plt.plot(base_fpr, mean_tprs2, 'b', label ='Filtro Casado Estocastico(AUC = %0.4f)' %(mean_auc))
    plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='blue', alpha=0.3,label ='$\pm$ 1. std. dev, (p = %0.3f, %0.3f)'  %(xFCE,yFCE))

    tprs_SS2 = np.array(tprs_SS)
    mean_SS = tprs_SS2.mean(axis=0)
    std_SS= tprs_SS2.std(axis=0) 
    SS_upper = np.minimum(mean_SS + std_SS, 1)
    SS_lower = mean_SS - std_SS

    mean_ac_SS = metrics.auc(base_fpr, mean_SS)

    xFC, yFC = p98(base_fpr, mean_SS)

    plt.plot(base_fpr, mean_SS, 'r', label ='Aproximacao filtro casado (AUC = %0.4f)' %(mean_ac_SS))
    plt.fill_between(base_fpr, SS_lower, SS_upper, color='red', alpha=0.3,label ='$\pm$ 1. std. dev, (p = %0.3f, %0.3f)'  %(xFC,yFC))

    p1=np.where(mean_tprs2<0.97)
    p1 = p1[0][-1]+1
    p2=np.where(mean_tprs2<0.998)
    p2 = p2[0][-1]+1
    plt.xlim([base_fpr[p1], base_fpr[p2]])
    plt.ylim([mean_tprs2[p1], mean_tprs2[p2]])
    #plt.xlim([0, 0.05])
    # plt.ylim([0.95, 1])
    plt.legend(loc = 'lower right')
    plt.ylabel('Taxa de Probabilidade de Detecção')
    plt.xlabel('Taxa de falso alarme')
    plt.title('Lado %d Modulo %d' %(side,mod))

    # data = [base_fpr, mean_tprs2, tprs_lower, tprs_upper, mean_auc, mean_SS, SS_lower, SS_upper, mean_ac_SS]
    filename = ('/content/drive/MyDrive/Filtro_casado_estocastico/Resultado_ROC_Canal/ROC_Parameters_side%d_mod%d.npy' %(side,mod))
    np.save(filename,data)

    plt.savefig('/content/drive/MyDrive/Filtro_casado_estocastico/Resultado_ROC_Canal/ROC_Side%d_mod%d.pdf' %(side,mod))
    plt.clf()

#     xFCE, yFCE = p98(base_fpr, mean_tprs2)
#     xFC, yFC = p98(base_fpr, mean_SS)

#     ar = [mean_auc, mean_ac_SS, xFCE, yFCE, xFC, yFC]

#     for ele in ar:
#       pontos.write(str(ele) + ',')
#     pontos.write('\n')



# pontos.close()
# files.download('pontos.txt') 


